# An Investigation in classifying the gender of players using their hours played and age

## INTRODUCTION

A research group in Computer Science at UBC, led by Frank Wood, collected data about how people play video games. They have set up a MineCraft server, and players' actions are recorded as they navigate through the world.

The research group gathered two datasets. The question we would want to answer with our project would be "Can we predict the gender if given the age and hours played on the MineCraft server?" So, we are only focusing on the first dataset, "players.csv", to answer our question.

The first dataset, players.csv, is a list of all unique players, including data about each player. In this dataset, there are 196 observations in total, and there are 7 variables for each observation. The variable “experience” is a categorical value which describes the player’s proficiency level in this video game, specifically, five levels: "Amateur", "Beginner", "Regular", "Veteran" and "Pro". The variable “subscribe” is a “lgl” value that identifies whether the player subscribed to MineCraft’s newsletter. The variable “hashedEmail” is a “chr” value which provides an encrypted email address from the player. The variable “played_hours” is a “dbl” value that gives the total time a player spent on the MineCraft server. The “name” variable is a “chr” value that has each player’s first name. The “gender” value is a categorical value which provides each player’s gender, specifically, 7 categories: “Agender”, “Female”, “Male”, “Non-binary”, “Two-Spirited”, “Other” and “Prefer not to say”. The variable “Age” is a quantitative variable that provides each individual player’s age. The potential issue that we want to clean and wrangle in this dataset before further steps would be the “NA” values in the “Age” column.


## DATA ANALYSIS

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [9]:
# loading the dataset players_csv
players <- read_csv("https://raw.githubusercontent.com/Ivy-Z-Z/DSCI100-010-group_project/refs/heads/main/players.csv")

# cleaning and wrangling

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
